In [13]:
import requests
import json
import pandas as pd
import datasets
import zipfile

In [16]:
zip_path = "../data/raw/website/redial_dataset.zip"

DATA_PATH = "../data/raw/website/train_data.jsonl"

### ダウンロード

In [15]:
# 1. zipファイルがあるフォルダのパスを取得 (../data/raw/website)
target_dir = os.path.dirname(zip_path)

# 2. そのフォルダを指定して解凍
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(path=target_dir)

# 3. パスを結合してファイルを読み込む
train_file_path = os.path.join(target_dir, "train_data.jsonl")
test_file_path = os.path.join(target_dir, "test_data.jsonl")

# trainデータの読み込み
train_data = []
# encoding='utf-8' を明示すると文字化けトラブルが減ります
with open(train_file_path, "r", encoding="utf-8") as f:
    for line in f:
        train_data.append(json.loads(line))
print("Loaded {} train conversations from {}".format(len(train_data), train_file_path))

# testデータの読み込み
test_data = []
with open(test_file_path, "r", encoding="utf-8") as f:
    for line in f:
        test_data.append(json.loads(line))
print("Loaded {} test conversations from {}".format(len(test_data), test_file_path))

Loaded 10006 train conversations from ../data/raw/website/train_data.jsonl
Loaded 1342 test conversations from ../data/raw/website/test_data.jsonl


In [17]:
def load_and_display_redial(file_path, limit=10):
    """
    ReDialデータを読み込み、会話内容と映画エンティティを整形して表示する
    """
    data_buffer = []
    
    # 1. 指定行数だけ読み込み
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= limit:
                break
            data_buffer.append(json.loads(line))
            
    print(f"Loaded {len(data_buffer)} conversations.\n")
    
    # 2. 整形して表示
    for i, conversation in enumerate(data_buffer):
        conv_id = conversation.get('conversationId', 'Unknown')
        initiator_id = conversation.get('initiatorWorkerId')
        movie_mentions = conversation.get('movieMentions', {})
        
        print(f"{'='*20} Conversation {i+1} (ID: {conv_id}) {'='*20}")
        
        # --- A. この会話に出てくる映画エンティティ (KG構築の正解データ) ---
        print(f"🎬 Movie Entities (Metadata):")
        if movie_mentions:
            for m_id, m_title in movie_mentions.items():
                print(f"  [{m_id}] : {m_title}")
        else:
            print("  (No movies tagged)")
            
        print("-" * 60)
        print("💬 Dialogue:")
        
        # --- B. 会話の流れ ---
        for msg in conversation.get('messages', []):
            sender_id = msg.get('senderWorkerId')
            text = msg.get('text', '').strip()
            
            # 役割の判定
            # initiator (会話を始めた人) = Seeker (映画を探しているユーザー)
            # others = Recommender (推薦システム役)
            if sender_id == initiator_id:
                role = "👤 USER (Seeker)"
            else:
                role = "🤖 SYSTEM (Rec.)"
            
            print(f"{role:<18}: {text}")
            
        print("\n")

In [18]:
# 実行して最初の10件を確認
load_and_display_redial(DATA_PATH, limit=10)

Loaded 10 conversations.

==================== Conversation 1 (ID: 391) ====================
🎬 Movie Entities (Metadata):
  [203371] : Final Fantasy: The Spirits Within (2001)
  [84779] : The Triplets of Belleville (2003)
  [122159] : Mary and Max (2009)
  [151313] : A Scanner Darkly  (2006)
  [191602] : Waking Life (2001)
  [165710] : The Boss Baby (2017)
------------------------------------------------------------
💬 Dialogue:
👤 USER (Seeker)   : Hi there, how are you? I'm looking for movie recommendations
🤖 SYSTEM (Rec.)   : I am doing okay. What kind of movies do you like?
👤 USER (Seeker)   : I like animations like @84779 and @191602
👤 USER (Seeker)   : I also enjoy @122159
👤 USER (Seeker)   : Anything artistic
🤖 SYSTEM (Rec.)   : You might like @165710 that was a good movie.
👤 USER (Seeker)   : What's it about?
🤖 SYSTEM (Rec.)   : It has Alec Baldwin it is about a baby that works for a company and gets adopted it is very funny
👤 USER (Seeker)   : That seems like a nice comedy
👤 USE